In [1]:
import subprocess
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = 'Arial'
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

In [3]:
filtered40_expr_df = pd.read_table('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/analysis_embryoExon_tissueExon/filtered_exons/filtered40_exonID_geneSymbols.tsv', 
             sep = "\t", )
filtered40_expr_df

,transcript_id,gene_id,gene_symbol,transcript_id_clean,gene_id_clean,exon_id,oocyte,1_cell,2_cell,4_cell,...,shield,7hpf,8hpf,bud,10.33hpf,16hpf,19hpf,20ss,total,max
0,ENSDART00000162026.2,ENSDARG00000098414.2,ninl,ENSDART00000162026,ENSDARG00000098414,ENSDART00000162026.2.16,0.147891,1.780893,0.197908,0.061905,...,0.124977,0.737046,0.199949,0.389906,0.185232,0.107895,0.268439,0.461453,20.733621,10.260420
1,ENSDART00000162026.2,ENSDARG00000098414.2,ninl,ENSDART00000162026,ENSDARG00000098414,ENSDART00000162026.2.9,0.000000,2.237812,0.000000,0.000000,...,0.005558,0.062000,0.000000,0.164775,0.000000,0.000000,0.024079,0.365155,8.935196,5.321738
2,ENSDART00000111774.3,ENSDARG00000077728.4,supv3l1,ENSDART00000111774,ENSDARG00000077728,ENSDART00000111774.3.8,0.082683,2.492953,0.043782,0.000000,...,0.190467,0.223202,0.217478,0.275413,0.197226,0.050224,0.051472,0.258635,13.874702,5.624056
3,ENSDART00000125257.3,ENSDARG00000077728.4,supv3l1,ENSDART00000125257,ENSDARG00000077728,ENSDART00000111774.3.8,0.082683,2.492953,0.043782,0.000000,...,0.190467,0.223202,0.217478,0.275413,0.197226,0.050224,0.051472,0.258635,13.874702,5.624056
4,ENSDART00000177841.2,ENSDARG00000109055.2,ftr81,ENSDART00000177841,ENSDARG00000109055,ENSDART00000177841.2.5,0.086045,6.088208,1.707729,0.337104,...,0.074549,0.066386,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.865714,14.167823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,ENSDART00000142233.2,ENSDARG00000094616.4,zgc:113984,ENSDART00000142233,ENSDARG00000094616,ENSDART00000176877.2.5_ENSDART00000190891.1.4_...,0.000000,3.405682,0.333384,0.000000,...,0.339051,0.000000,0.000000,0.172485,0.000000,0.000000,0.000000,0.000000,31.327701,13.301727
260,ENSDART00000142233.2,ENSDARG00000094616.4,zgc:113984,ENSDART00000142233,ENSDARG00000094616,ENSDART00000133880.3.9,0.000000,15.918922,0.000000,0.000000,...,0.077139,0.000000,0.000000,0.046296,0.000000,0.000000,0.000000,0.000000,35.024673,15.918922
261,ENSDART00000145657.2,ENSDARG00000094616.4,zgc:113984,ENSDART00000145657,ENSDARG00000094616,ENSDART00000144851.3.8_ENSDART00000133880.3.8_...,0.000000,8.884253,0.428189,0.000000,...,0.280748,0.000000,0.000000,0.057247,0.000000,0.000000,0.000000,0.000000,20.610753,8.884253
262,ENSDART00000145657.2,ENSDARG00000094616.4,zgc:113984,ENSDART00000145657,ENSDARG00000094616,ENSDART00000176877.2.5_ENSDART00000190891.1.4_...,0.000000,3.405682,0.333384,0.000000,...,0.339051,0.000000,0.000000,0.172485,0.000000,0.000000,0.000000,0.000000,31.327701,13.301727


In [58]:
exon_gtf = pd.read_table('/mnt/Storage/home/wangyiman/source/bySpecies/danRer11_2/ensGene/danRer11_2.ensGene.ExonUniq.gtf',
                        sep = "\t", header = None, usecols = [0,3,4,5,7,6,8])

In [59]:
exon_gtf[5] = exon_gtf.iloc[:,6].str.split('"', expand = True)[1]

In [60]:
exon_gtf = exon_gtf.drop(8, axis = 1).drop_duplicates()
exon_gtf = exon_gtf[[0,3,4,5,7,6]]
exon_gtf

,0,3,4,5,7,6
0,chr1,6408,6760,ENSDART00000164359.1.1,.,-
2,chr1,6892,6955,ENSDART00000164359.1.2_ENSDART00000157701.1.2_...,.,-
4,chr1,9558,9694,ENSDART00000164359.1.3_ENSDART00000167898.1.3,.,-
6,chr1,10081,10191,ENSDART00000164359.1.4_ENSDART00000167898.1.4,.,-
8,chr1,11550,11625,ENSDART00000164359.1.5,.,-
...,...,...,...,...,...,...
644820,chr4,29904416,29904496,ENSDART00000181414.1_50.1_ENSDART00000186439.1...,.,+
644822,chr4,30480489,30480569,ENSDART00000181414.1_51.1_ENSDART00000186439.1...,.,-
644824,chr4,29920655,29920735,ENSDART00000181414.1_52.1_ENSDART00000185948.1...,.,+
644826,chr4,30483454,30483534,ENSDART00000181414.1_53.1_ENSDART00000186439.1...,.,-


In [61]:
exon_gtf.columns = ['chr', 'start', 'end', 'exon_id', 'score', 'strand']

In [65]:
filtered40_exon_bed = pd.merge(exon_gtf, filtered40_expr_df[['gene_symbol','transcript_id_clean','gene_id_clean','exon_id']], on = 'exon_id')
filtered40_exon_bed['start'] = filtered40_exon_bed['start'] - 1
filtered40_exon_bed['exon_id'] = 'exon_id:' + filtered40_exon_bed['exon_id'] + '|' + 'gene_symbol:' + filtered40_exon_bed['gene_symbol']
filtered40_exon_bed

,chr,start,end,exon_id,score,strand,gene_symbol,transcript_id_clean,gene_id_clean
0,chr13,7186676,7186746,exon_id:ENSDART00000162026.2.9|gene_symbol:ninl,.,-,ninl,ENSDART00000162026,ENSDARG00000098414
1,chr13,7197591,7197702,exon_id:ENSDART00000162026.2.16|gene_symbol:ninl,.,-,ninl,ENSDART00000162026,ENSDARG00000098414
2,chr13,23045553,23045645,exon_id:ENSDART00000111774.3.8|gene_symbol:sup...,.,-,supv3l1,ENSDART00000111774,ENSDARG00000077728
3,chr13,23045553,23045645,exon_id:ENSDART00000111774.3.8|gene_symbol:sup...,.,-,supv3l1,ENSDART00000125257,ENSDARG00000077728
4,chr13,33464474,33464708,exon_id:ENSDART00000177841.2.4|gene_symbol:ftr81,.,+,ftr81,ENSDART00000177841,ENSDARG00000109055
...,...,...,...,...,...,...,...,...,...
259,chr9,8393261,8393424,exon_id:ENSDART00000133880.3.9|gene_symbol:zgc...,.,+,zgc:113984,ENSDART00000190891,ENSDARG00000094616
260,chr9,8393261,8393424,exon_id:ENSDART00000133880.3.9|gene_symbol:zgc...,.,+,zgc:113984,ENSDART00000144851,ENSDARG00000094616
261,chr9,8393261,8393424,exon_id:ENSDART00000133880.3.9|gene_symbol:zgc...,.,+,zgc:113984,ENSDART00000133880,ENSDARG00000094616
262,chr9,8393261,8393424,exon_id:ENSDART00000133880.3.9|gene_symbol:zgc...,.,+,zgc:113984,ENSDART00000142233,ENSDARG00000094616


In [66]:
filtered40_exon_bed.to_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/analysis_embryoExon_tissueExon/filtered_exons/filtered40_anno_exonID_geneSymbols.bed',
                          sep = "\t", index = False, header = False)

In [67]:
%%bash
cd ~/maternal_loading/2.public_data/analysis_embryoExon_tissueExon/filtered_exons
bedtools getfasta -fi ~/source/bySpecies/danRer11_2/fasta/danRer11_2_main.fa -bed filtered40_anno_exonID_geneSymbols.bed -s -name | fold -w 100 > filtered40_exon.fasta
bedtools getfasta -fi ~/source/bySpecies/danRer11_2/fasta/danRer11_2_main.fa -bed filtered40_anno_exonID_geneSymbols.bed -s -name -tab > filtered40_exon_seq.tsv
